In [1]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav

# download and load all models
preload_models()

No GPU being used. Careful, inference might be very slow!
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /home/zaibachkhoa/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████████████████████████████████| 88.9M/88.9M [00:08<00:00, 11.1MB/s]


In [3]:
# https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing#scrollTo=cWuW8SdgcO_Q
from transformers import AutoProcessor, BarkModel
import torch, os
import scipy

device = "cuda:0" if torch.cuda.is_available() else "cpu"
CUR_PATH = "/home/zaibachkhoa/Documents/bark/"  # os.path.dirname(os.path.abspath(__file__))
# default_cache_dir = os.path.join(os.path.expanduser("~"), ".cache")
CACHE_DIR = os.path.join(CUR_PATH, "bark", "loa_mod")
os.environ['HF_HOME'] = CACHE_DIR
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR
os.environ['PYTORCH_PRETRAINED_BERT_CACHE'] = CACHE_DIR

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")
# model = BarkModel.from_pretrained("suno/bark-small")
model = model.to(device)

voice_preset = "v2/en_speaker_5"

/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

In [4]:
# Py_Bark
text_prompt = "Hanoi is nicest at the night"
text_prompt = f"♪ {text_prompt} ♪"
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("Py_Bark.wav", SAMPLE_RATE, audio_array)

100%|███████████████████████████████████████████| 14/14 [00:56<00:00,  4.03s/it]


In [5]:
# Py_Transformers
inputs = processor(
    """
         ♪ In the jungle, the mighty jungle, the lion barks tonight ♪
    """
)
# inputs = processor("[clears throat] Hello uh ..., my dog is cute [laughter]")
# # https://sunoaiwiki.com/en/resources/2024-05-13-list-of-metatags/

audio_array = model.generate(
    **inputs.to(device),
    # num_beams=4,
    # temperature=0.5,
    # semantic_temperature=0.8,
)
audio_array = audio_array.cpu().numpy().squeeze()

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("Py_Transformers.wav", rate=sample_rate, data=audio_array)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
